<b>Libraries used in this analysis</b>

In [ ]:
#libraries used in phase 1:
from siphon.simplewebservice.ndbc import NDBC #UNIDATA library - https://github.com/Unidata/siphon 
import pandas as pd

#libraries used in phase 2:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.colors import BoundaryNorm
import matplotlib.pyplot as plt
import numpy as np # this libary will be used in phase 2 in tandem with interpolation commands

from metpy.interpolate import interpolate_to_grid, remove_nan_observations #https://unidata.github.io/MetPy/latest/api/generated/metpy.interpolate.html
from metpy.units import units

to_proj = ccrs.AlbersEqualArea(central_longitude=-97., central_latitude=38.)

<b>Description of new Unidata Libraries used in this script:</b>
<ul>
<li>Siphon - Siphon is a collection of Python utilities for downloading data from Unidata data technologies. See our support page for ways to get help with Siphon.</li>
<li>metpy.interpolate - a collection of metpy library tools used for interpolating data</li>
</ul>

<b>Phase I - Follows Video #54 - time series analysis</b>

#### Step 1 - Load the data using Siphon - here I am loading all of the observations for one bouy on Lake Ontario, near Oswego New York
Data Source: https://www.ndbc.noaa.gov/

In [ ]:
s_bouy = NDBC.realtime_observations('RPRN6')

In [ ]:
s_bouy.head()

In [ ]:
s_bouy.info()

Note: Notice that time field. 

#### Step 2 - make a copy of the time field

In [ ]:
s_bouy['time_copy'] = s_bouy['time']

In [ ]:
s_bouy.info()

#### Step 3 - cast the time field to numeric. Why is this done? Try running the code below with time and and time_copy. 

In [ ]:
s_bouy['time'] = pd.to_numeric(s_bouy['time'], errors='coerce')

#### Step 4 - Create a small multiple chart for different collected data points from the Bouy

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 10)) #this is making a three-row plot with one column of data - referred to as small multiples
ax2b = ax2.twinx() #Minute 8:45 - this will allow for a plot to have two y - axis sharing the same x-axis

#Pressure
ax1.plot(s_bouy['time'], s_bouy['pressure'], color='black')
ax1.set_ylabel('Pressure [hPa]')

#Wind Speed, gust and direction
ax2.plot(s_bouy['time'], s_bouy['wind_speed'], color = 'tab:orange')
ax2.plot(s_bouy['time'], s_bouy['wind_gust'], color = 'tab:olive', linestyle='--')
ax2.plot(s_bouy['time'], s_bouy['wind_direction'], color = 'tab:blue', linestyle='-')
ax2.set_ylabel('Wind Speed [m\s]')
ax2b.set_ylabel('Wind Directon')

#Water Temperature
ax3.plot(s_bouy['time'], s_bouy['water_temperature'], color = 'tab:brown')
ax3.set_ylabel('Water Temp [degC]')





<b>Phase II - Plot Temerpature Isotherms from NDBC latest observations</b>

#### Step 1 - download the latest_observations dataset for all NDBC bouys

In [ ]:
data_aval = NDBC.latest_observations

In [ ]:
print(data_aval)

#### Step 2 - Set the NDBC bouy observations to a dataframe obejct and inspect using, head(), tail(), and info()

In [ ]:
b_df = NDBC.latest_observations()

In [ ]:
b_df.head()

#### Step 2a - Optional - this command will allow ALL of the records in a dataframe display. 

In [ ]:
pd.set_option('display.max_rows', b_df.shape[0]+1)

In [ ]:
b_df

In [ ]:
b_df.info()

#### Step 3 - Export this captured data to a CSV file <br>
#### Note: this is done so that we can actually manipulate the data that has been captured. 

In [ ]:
b_df.to_csv(r'C:\Unidata\Bouy3_02-20b.csv')

In [ ]:
b2a_df = pd.read_csv("C:/Unidata/Bouy3_02-20b.csv",na_values="NaN", keep_default_na=False)

In [ ]:
b2a_df.tail()

<b>Note that there are NaN values included in this imported data</b>

#### Step 4 - replace all of the blank values with zero in the air_temperature field

In [ ]:
b2a_df['air_temperature'].replace(to_replace='', value=0, inplace=True)

In [ ]:
b2a_df.tail()

#### Step 5 - drop unneeded fields from the analysis

In [ ]:
b3a_df = b2a_df.drop(b2a_df.loc[:, 'wind_direction':'3hr_pressure_tendency'].columns, axis = 1) 

In [ ]:
b3a_df.head()

In [ ]:
b4a_df = b3a_df.drop(b3a_df.loc[:, 'water_temperature':'time'].columns, axis = 1)

In [ ]:
b4a_df.head()

In [ ]:
b4a_df.info()

#### Step 6 - convert the air_temperature to a numeric value

In [ ]:
b4a_df['air_temperature'] = pd.to_numeric(b4a_df['air_temperature'], errors='coerce')

##### Check to make sure air_temperature is now a numeric value

In [ ]:
b4a_df.info()

#### Step 7 - Prepare the bouy data for interpolation and plotting on the maps <br>
#### Source code: https://unidata.github.io/MetPy/latest/examples/gridding/Wind_SLP_Interpolation.html#sphx-glr-examples-gridding-wind-slp-interpolation-py

In this step we are assinging the long/lat pairs to objects and preparing them for plotting

In [ ]:
lon = b4a_df['longitude'].values
lat = b4a_df['latitude'].values
xp, yp, _ = to_proj.transform_points(ccrs.Geodetic(), lon, lat).T

Next, we are cleaning the air_temperature observations and interpolating the bouy readings to create a continuous surface of air temperature values using metpy interpolate

In [ ]:
x_masked, y_masked, t = remove_nan_observations(xp, yp, b4a_df['air_temperature'].values)
tempx, tempy, temp = interpolate_to_grid(x_masked, y_masked, t, interp_type='cressman',
                                         minimum_neighbors=3, search_radius=400000, hres=35000)

temp = np.ma.masked_where(np.isnan(temp), temp)


In [ ]:
levels = list(range(-20, 20, 1))
cmap = plt.get_cmap('viridis')
norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)

fig = plt.figure(figsize=(20, 10))
#add_metpy_logo(fig, 360, 120, size='large')
view = fig.add_subplot(1, 1, 1, projection=to_proj)

view.set_extent([-120, -70, 20, 50])
#view.add_feature(cfeature.STATES.with_scale('50m'))
view.add_feature(cfeature.OCEAN)
view.add_feature(cfeature.COASTLINE.with_scale('50m'))
view.add_feature(cfeature.BORDERS, linestyle=':')

cs = view.contour(tempx, tempy, temp, colors='k', levels=list(range(-20, 20, 2)))
view.clabel(cs, inline=1, fontsize=12, fmt='%i')

mmb = view.pcolormesh(tempx, tempy, temp, cmap=cmap, norm=norm)
fig.colorbar(mmb, shrink=.4, pad=0.02, boundaries=levels)

#view.barbs(windgridx, windgridy, uwind, vwind, alpha=.4, length=5)

view.set_title('Air Temperature from Bouy Data March 2, 2020')

plt.show()

<b>Resources and code bases used:</b>
 <ul>
   <p>Bouy Data Source</p><li>https://www.ndbc.noaa.gov/</li>
  <p>Interpolation and Source Code Used</p><li>https://unidata.github.io/MetPy/latest/examples/gridding/Wind_SLP_Interpolation.html#sphx-glr-examples-gridding-wind-slp-interpolation-py</li>
  <p>Code that follows the video</p><li>https://unidata.github.io/python-training/workshop/Time_Series/basic-time-series-plotting/#loaddata</li>
    <p>Mety Py Mondays Video Inspiration</p><li><p>#55</p>https://www.youtube.com/watch?v=SPe8jYOURfc</li><li><p>#54</p>https://www.youtube.com/watch?v=ovqlYlI7l5A</li>
</ul> 
